In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Exploration

In [3]:
us_crime_data = pd.read_excel("/content/Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2018.xls")
us_crime_data.head()


,Table 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,by State by Metropolitan and Nonmetropolitan C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[The data shown in this table do not reflect c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
4,ALABAMA - Metropolitan Counties,Autauga,51,0,6,5,40,372,92,240,40,NaN


In [4]:
us_crime_data.tail(10)

,Table 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
2359,NaN,Washakie,0,0,0,0,0,17,4,13,0,0
2360,NaN,Weston,5,0,1,0,4,0,0,0,0,0
2361,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2362,2 The FBI does not publish arson data unless i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2363,3 The FBI determined that the agency's data we...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2364,4 The Tulare County Highway Patrol collects th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2365,5 Because of changes in the state/local agency...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2366,6 Limited data for 2018 were available for Iowa.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2367,7 This agency/state submits rape data classifi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2368,8 The FBI determined that the agency's data we...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning

#### Change Column Names

In [5]:
header_row = 3
us_crime_data.columns = us_crime_data.iloc[header_row]
us_crime_data.head()

3,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
0,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,by State by Metropolitan and Nonmetropolitan C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[The data shown in this table do not reflect c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
4,ALABAMA - Metropolitan Counties,Autauga,51,0,6,5,40,372,92,240,40,NaN


In [6]:
us_crime_data.columns = us_crime_data.columns.str.replace("\n", " ")

In [7]:
us_crime_data = us_crime_data.rename(columns={"Arson2":"Arson"})

#### Delete first 4 rows

In [8]:
us_crime_data = us_crime_data.iloc[4:]

#### Delete last 8 rows

In [9]:
us_crime_data = us_crime_data.drop(us_crime_data.tail(8).index)
us_crime_data.tail(5)

3,State,County,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
2356,NaN,Sublette,4,0,0,0,4,49,11,35,3,0
2357,NaN,Sweetwater,22,0,8,0,14,77,17,52,8,0
2358,NaN,Uinta,7,1,1,1,4,53,12,35,6,0
2359,NaN,Washakie,0,0,0,0,0,17,4,13,0,0
2360,NaN,Weston,5,0,1,0,4,0,0,0,0,0


### Forward Fill State Names

In [10]:
us_crime_data['State'] = us_crime_data['State'].fillna(method = 'ffill')

In [11]:
new = us_crime_data['State'].str.split(" -", n = 1, expand = True)

In [12]:
new

,0,1
4,ALABAMA,Metropolitan Counties
5,ALABAMA,Metropolitan Counties
6,ALABAMA,Metropolitan Counties
7,ALABAMA,Metropolitan Counties
8,ALABAMA,Metropolitan Counties
...,...,...
2356,WYOMING,Nonmetropolitan Counties
2357,WYOMING,Nonmetropolitan Counties
2358,WYOMING,Nonmetropolitan Counties
2359,WYOMING,Nonmetropolitan Counties


In [13]:
us_crime_data['State'] = new[0]

In [14]:
us_crime_data['State'] = us_crime_data['State'].str.replace('IOWA6', 'IOWA')

### Separate column with State Abbreviations

In [15]:
us_state_abbrev = {
    'ALABAMA': 'AL', 
    'ALASKA': 'AK', 
    'ARIZONA': 'AZ', 
    'ARKANSAS': 'AR', 
    'CALIFORNIA': 'CA', 
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT', 
    'DELAWARE': 'DE', 
    'FLORIDA': 'FL', 
    'GEORGIA': 'GA', 
    'HAWAII': 'HI', 
    'IDAHO': 'ID',
    'ILLINOIS': 'IL', 
    'INDIANA': 'IN', 
    'IOWA': 'IA', 
    'KANSAS': 'KS', 
    'KENTUCKY': 'KY', 
    'LOUISIANA': 'LA',
    'MAINE': 'ME', 
    'MARYLAND': 'MD', 
    'MASSACHUSETTS': 'MA', 
    'MICHIGAN': 'MI', 
    'MINNESOTA': 'MN', 
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO', 
    'MONTANA': 'MT', 
    'NEBRASKA': 'NE', 
    'NEVADA': 'NV', 
    'NEW HAMPSHIRE': 'NH', 
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM', 
    'NEW YORK': 'NY', 
    'NORTH CAROLINA': 'NC', 
    'NORTH DAKOTA': 'ND', 
    'OHIO': 'OH', 
    'OKLAHOMA': 'OK',
    'OREGON': 'OR', 
    'PENNSYLVANIA': 'PA', 
    'RHODE ISLAND': 'RI', 
    'SOUTH CAROLINA': 'SC', 
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN', 
    'TEXAS': 'TX', 
    'UTAH': 'UT', 
    'VERMONT': 'VT', 
    'VIRGINIA': 'VA', 
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV', 
    'WISCONSIN': 'WI', 
    'WYOMING': 'WY'}

In [16]:
us_crime_data['State Abbreviations'] = us_crime_data['State'].map(us_state_abbrev)

In [17]:
us_crime_data.head()

3,State,County,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,State Abbreviations
4,ALABAMA,Autauga,51,0,6,5,40,372,92,240,40,NaN,AL
5,ALABAMA,Baldwin,223,0,9,37,177,615,173,397,45,NaN,AL
6,ALABAMA,Blount,375,1,19,5,350,796,191,492,113,NaN,AL
7,ALABAMA,Calhoun,14,0,5,7,2,144,49,95,0,NaN,AL
8,ALABAMA,Elmore,68,4,30,12,22,669,178,427,64,NaN,AL


## NaN?

In [18]:
us_crime_data.isnull().sum()

3
State                                     0
County                                    1
Violent crime                            61
Murder and nonnegligent manslaughter      2
Rape1                                    56
Robbery                                   2
Aggravated assault                        4
Property crime                            9
Burglary                                  8
Larceny- theft                            1
Motor vehicle theft                       2
Arson                                   124
State Abbreviations                       0
dtype: int64

In [19]:
empty = ["Violent crime", "Murder and nonnegligent manslaughter", "Rape1", "Robbery", "Aggravated assault", "Property crime", "Burglary", "Larceny- theft", "Motor vehicle theft", "Arson"]
us_crime_data[empty] = us_crime_data[empty].fillna(value=0)

In [20]:
us_crime_data.isnull().sum()

3
State                                   0
County                                  1
Violent crime                           0
Murder and nonnegligent manslaughter    0
Rape1                                   0
Robbery                                 0
Aggravated assault                      0
Property crime                          0
Burglary                                0
Larceny- theft                          0
Motor vehicle theft                     0
Arson                                   0
State Abbreviations                     0
dtype: int64

### Reorder Columns

In [21]:
us_crime_data.columns

Index(['State', 'County', 'Violent crime',
       'Murder and nonnegligent manslaughter', 'Rape1', 'Robbery',
       'Aggravated assault', 'Property crime', 'Burglary', 'Larceny- theft',
       'Motor vehicle theft', 'Arson', 'State Abbreviations'],
      dtype='object', name=3)

In [22]:
us_crime_data = us_crime_data[['State', 'State Abbreviations', 'County', 'Violent crime',
       'Murder and nonnegligent manslaughter', 'Rape1', 'Robbery',
       'Aggravated assault', 'Property crime', 'Burglary', 'Larceny- theft',
       'Motor vehicle theft', 'Arson']]

### Update County Names

In [23]:
us_crime_data['County'] = us_crime_data['County'].str.replace(' County Police Department', '')
us_crime_data['County'] = us_crime_data['County'].str.replace(' Police Department', '')
us_crime_data['County'] = us_crime_data['County'].str.replace(' County', '')
us_crime_data['County'] = us_crime_data['County'].str.replace('7', '')
us_crime_data['County'] = us_crime_data['County'].str.replace('5', '')
us_crime_data['County'] = us_crime_data['County'].str.replace('3', '')
us_crime_data['County'] = us_crime_data['County'].str.replace("'s", '')

In [24]:
# Merge counties together after name cleaning
#https://stackoverflow.com/questions/48115481/pandas-combine-rows-based-on-certain-column-values-and-nan

In [25]:
us_crime_data = us_crime_data.groupby(['State', 'State Abbreviations', 'County'])["Violent crime", "Murder and nonnegligent manslaughter", "Rape1", "Robbery", "Aggravated assault", "Property crime", "Burglary", "Larceny- theft", "Motor vehicle theft", "Arson"].sum().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


#### Check 

In [26]:
us_crime_data.head()

3,State,State Abbreviations,County,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,ALABAMA,AL,Autauga,51,0,6,5,40,372,92,240,40,0
1,ALABAMA,AL,Baldwin,223,0,9,37,177,615,173,397,45,0
2,ALABAMA,AL,Blount,375,1,19,5,350,796,191,492,113,0
3,ALABAMA,AL,Bullock,7,0,0,3,4,42,23,17,2,0
4,ALABAMA,AL,Calhoun,14,0,5,7,2,144,49,95,0,0


In [27]:
us_crime_data.tail()

3,State,State Abbreviations,County,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
2333,WYOMING,WY,Sublette,4,0,0,0,4,49,11,35,3,0
2334,WYOMING,WY,Sweetwater,22,0,8,0,14,77,17,52,8,0
2335,WYOMING,WY,Uinta,7,1,1,1,4,53,12,35,6,0
2336,WYOMING,WY,Washakie,0,0,0,0,0,17,4,13,0,0
2337,WYOMING,WY,Weston,5,0,1,0,4,0,0,0,0,0


In [28]:
us_crime_data.to_csv('us_crime_data.csv')

In [29]:
!cp us_crime_data.csv "drive/My Drive/"

## Split Dataset into State CSV's

In [30]:
import os

In [31]:
for i, x in us_crime_data.groupby('State'):
    p = os.path.join(os.getcwd(), "crime_{}.csv".format(i.lower()))
    x.to_csv(p, index=False)